In [ ]:
#| default_exp 16_ngame-linker-for-amazontitles

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import os,torch,json, torch.multiprocessing as mp, joblib, numpy as np, scipy.sparse as sp

from xcai.basics import *
from xcai.models.PPP0XX import DBT009,DBT011

In [ ]:
os.environ['WANDB_MODE'] = 'disabled'

In [ ]:
#| export
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
os.environ['WANDB_PROJECT'] = 'mogicX_03-amazontitles-linker'

## Driver

In [ ]:
#| export
if __name__ == '__main__':
    output_dir = '/scratch/scai/phd/aiz218323/outputs/mogicX/16_ngame-linker-for-amazontitles'

    data_dir = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/'
    config_file = 'amazontitles'
    config_key = 'data_meta'

    mname = 'sentence-transformers/msmarco-distilbert-base-v4'

    input_args = parse_args()

    pkl_file = f'{input_args.pickle_dir}/mogicX/amazontitles_data-meta_distilbert-base-uncased'
    pkl_file = f'{pkl_file}_sxc' if input_args.use_sxc_sampler else f'{pkl_file}_xcs'
    if input_args.only_test: pkl_file = f'{pkl_file}_only-test'
    pkl_file = f'{pkl_file}.joblib'

    do_inference = input_args.do_train_inference or input_args.do_test_inference or input_args.save_train_prediction or input_args.save_test_prediction or input_args.save_representation

    os.makedirs(os.path.dirname(pkl_file), exist_ok=True)
    block = build_block(pkl_file, config_file, input_args.use_sxc_sampler, config_key, do_build=input_args.build_block, only_test=input_args.only_test, 
                        data_dir=data_dir)

    linker_block = block.linker_dset('cat_meta', remove_empty=True)

    args = XCLearningArguments(
        output_dir=output_dir,
        logging_first_step=True,
        per_device_train_batch_size=800,
        per_device_eval_batch_size=800,
        representation_num_beams=200,
        representation_accumulation_steps=10,
        save_strategy="steps",
        evaluation_strategy="steps",
        eval_steps=5000,
        save_steps=5000,
        save_total_limit=5,
        num_train_epochs=300,
        predict_with_representation=True,
        representation_search_type='BRUTEFORCE',
        adam_epsilon=1e-6,
        warmup_steps=100,
        weight_decay=0.01,
        learning_rate=2e-4,
    
        group_by_cluster=True,
        num_clustering_warmup_epochs=10,
        num_cluster_update_epochs=5,
        num_cluster_size_update_epochs=25,
        clustering_type='EXPO',
        minimum_cluster_size=2,
        maximum_cluster_size=1600,
    
        metric_for_best_model='P@1',
        load_best_model_at_end=True,
        target_indices_key='plbl2data_idx',
        target_pointer_key='plbl2data_data2ptr',
    
        use_encoder_parallel=True,
        max_grad_norm=None,
        fp16=True,
    )

    def model_fn(mname, bsz):
        model = DBT009.from_pretrained(mname, bsz=bsz, tn_targ=5000, margin=0.3, tau=0.1, n_negatives=10, 
                                       apply_softmax=True, use_encoder_parallel=True)
        return model
    
    def init_fn(model): 
        model.init_dr_head()

    metric = PrecReclMrr(linker_block.n_lbl, linker_block.test.data_lbl_filterer, prop=linker_block.train.dset.data.data_lbl,
                         pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200], mk=[5, 10, 20])
    
    bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

    model = load_model(args.output_dir, model_fn, {"mname": mname, "bsz": bsz}, init_fn, do_inference=do_inference, use_pretrained=input_args.use_pretrained)
    
    learn = XCLearner(
        model=model,
        args=args,
        train_dataset=linker_block.train.dset,
        eval_dataset=linker_block.test.dset,
        data_collator=linker_block.collator,
        compute_metrics=metric,
    )
    
    main(learn, input_args, n_lbl=linker_block.n_lbl)
    

## Predictions

In [ ]:
from xcai.analysis import *
from xclib.utils.sparse import retain_topk

In [ ]:
output_dir = '/scratch/scai/phd/aiz218323/outputs/mogicX/16_ngame-linker-for-amazontitles'

data_dir = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/'
config_file = 'amazontitles'
config_key = 'data_meta'

pickle_dir = '/scratch/scai/phd/aiz218323/datasets/processed/'

In [ ]:
pkl_file = f'{pickle_dir}/mogicX/amazontitles_data-meta_distilbert-base-uncased_sxc.joblib'
os.makedirs(os.path.dirname(pkl_file), exist_ok=True)
block = build_block(pkl_file, config_file, True, config_key, data_dir=data_dir)

In [ ]:
linker_block = block.linker_dset('cat_meta', remove_empty=False)

In [ ]:
pred_dir = '/home/scai/phd/aiz218323/scratch/outputs/mogicX/16_ngame-linker-for-amazontitles-001/predictions/'
pred_lbl = sp.load_npz(f'{pred_dir}/test_predictions_full.npz')

pred_block = get_pred_dset(retain_topk(pred_lbl, k=5), linker_block.test.dset)

disp = CompareDataset(XCDataset._initialize(linker_block.test.dset), pred_block, '1. ', '2. ')

In [ ]:
metric = pointwise_eval(pred_lbl, linker_block.test.dset.data.data_lbl, topk=5)

In [ ]:
idxs = np.argsort(np.ravel(metric.sum(axis=1)))[:-100-1:-1]
disp.show(idxs)

1.  data_input_text : Bella Ladies Sophie Sheer Rib Long Sleeve T-Shirt. 8751
2.  data_input_text : Bella Ladies Sophie Sheer Rib Long Sleeve T-Shirt. 8751
1.  lbl2data_input_text : ['Clothing, Shoes & Jewelry', 'Novelty, Costumes & More', 'Sports & Outdoors', 'Clothing', 'Women', 'Band & Music Fan', 'Shirts', 'Tops & Tees', 'T-Shirts', 'B', 'Knits & Tees', 'Bella']
2.  lbl2data_input_text : ['Clothing, Shoes & Jewelry', 'Clothing', 'Women', 'T-Shirts', 'Bella']

1.  data_input_text : ecko unltd. Men's Herbaceous Skull Shirt, Atash, XX-Large
2.  data_input_text : ecko unltd. Men's Herbaceous Skull Shirt, Atash, XX-Large
1.  lbl2data_input_text : ['Clothing, Shoes & Jewelry', 'Clothing', 'Men', 'Shirts', 'T-Shirts', 'E', 'Ecko Unltd.']
2.  lbl2data_input_text : ['Clothing', 'Men', 'Shirts', 'T-Shirts', 'Ecko Unltd.']

1.  data_input_text : Dyeables Women's Brielle Ankle-Strap Sandal
2.  data_input_text : Dyeables Women's Brielle Ankle-Strap Sandal
1.  lbl2data_input_text : ['Clothing, S

In [ ]:
idxs = np.argsort(np.ravel(metric.sum(axis=1)))[:100]
disp.show(idxs)

1.  data_input_text : Everyday Italian With Giada De Laurentis V2: Anytime Italian
2.  data_input_text : Everyday Italian With Giada De Laurentis V2: Anytime Italian
1.  lbl2data_input_text : []
2.  lbl2data_input_text : ['Books', 'Medical Books', 'Book of the Dead (Egyptian)', 'Art Books', 'Book of the Dead (Tibetan)']

1.  data_input_text : Noddy and Tessie Bear
2.  data_input_text : Noddy and Tessie Bear
1.  lbl2data_input_text : ['Books']
2.  lbl2data_input_text : ['Teddy Bears', 'Bears', 'Annie', 'Nick & Nora', 'JP and Mattie']

1.  data_input_text : Meet My Cats
2.  data_input_text : Meet My Cats
1.  lbl2data_input_text : ['Books']
2.  lbl2data_input_text : ['Cats', 'Cats, Dogs & Animals', 'All Cat', 'Travel with Pets', 'Boy Meets Girl']

1.  data_input_text : Lift Him Up With Ron Kenoly [VHS]
2.  data_input_text : Lift Him Up With Ron Kenoly [VHS]
1.  lbl2data_input_text : ['CDs & Vinyl', 'Christian', 'Pop & Contemporary', 'Praise & Worship', 'Gospel']
2.  lbl2data_input_text : 